### **Generación de Embeddings**

Este notebook se encargará de generar embeddings para cada producto utilizando sentence-transformers y almacenarlos en Elasticsearch. Además, se genera un respaldo del DataFrame con embeddings  para futuras consultas o análisis.

**Librerias**

In [1]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch, helpers
import json

**Conexion y carga**

In [ ]:
#  Conectar a Elasticsearch
es = Elasticsearch("http://localhost:9200")
index_name = "amazon_products"

#  Cargar el modelo preentrenado
# Se deja de usar por el alto tiempo de procesamiento, se puede usar esta opcion con recursos elevados de procesamiento
#model = SentenceTransformer('all-mpnet-base-v2')

#  Cargar el modelo optimizado
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

#  Cargar de df procesado
df_path = "../data/processed/df_pro.csv" 
df = pd.read_csv(df_path)

A continuación generaremos los embeddings usando el modelo, para asegurarse de no tener problemas de recursos se usará batch_size. En las ejecuciones de prueba el modelo optimizado tomó 7 min en ejecutarse.

**Si se descarga el archivo parquet con los embeddings ya generados y se los coloca en la carpeta correspondiente, se puede saltar hasta la celda 6 de este nb.**

In [3]:
# Convertir la columna 'name' a lista
product_names = df['name'].tolist()
# Generar embeddings en batches para optimizar
batch_size = 128  
embeddings = model.encode(product_names, batch_size=batch_size, show_progress_bar=True)
df['embedding'] = [emb.tolist() for emb in embeddings]

Batches:   0%|          | 0/3836 [00:00<?, ?it/s]

c:\Users\franj\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


**Comprobando embeddings generados antes de indexarlos**

In [4]:
# Comprobación de generacion de embedding
ejemplo_embedding = df['embedding'].iloc[10]
print(f"Ejemplo de embedding:\n{ejemplo_embedding[:10]}...")  # Mostrar los primeros 10 números
print(f"Tamaño del embedding: {len(ejemplo_embedding)}")


Ejemplo de embedding:
[-0.1474011242389679, -0.0034425125923007727, -0.0014247492654249072, 0.02615339495241642, 0.028749532997608185, -0.062161415815353394, -0.02965972200036049, 0.025048580020666122, -0.07818062603473663, -0.05018840357661247]...
Tamaño del embedding: 384


In [5]:
# Guardar el DataFrame con embeddings en un archivo parquet
df.to_parquet("../data/processed/amazon_products_with_embeddings.parquet")
print("Embeddings guardados exitosamente.")

Embeddings guardados exitosamente.


In [ ]:
##Codigo para cargar embeddings y no generarlos
df= pd.read_parquet("../data/processed/amazon_products_with_embeddings.parquet")
# Convertir cada embedding a lista
df['embedding'] = df['embedding'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

**Reduccion de dimensiones(opcional)(no se usó en app.py)**

Se desarrolló una función para reducir la dimensionalidad del primer modelo que se probó (all-mpnet-base-v2), pero se decidió que era una mejor estrategia cambiar de modelo a uno más ligero para reducir tiempos de generación e indexación.

import umap.umap_ as umap

def aplicar_umap(df, n_dim=200):
    """
    Reduce la dimensionalidad de los embeddings usando UMAP.

    Args:
        df (pd.DataFrame): DataFrame con la columna 'embedding' que contiene los embeddings originales.
        n_dim (int): Número de dimensiones deseadas tras la reducción (por defecto 200).
        
    Returns:
        pd.DataFrame: DataFrame con la columna 'embedding' actualizada con los embeddings reducidos.
    """
    # Convertir los embeddings a un array NumPy
    embeddings = np.array(df['embedding'].tolist())
    
    ##embeddings=df['embedding']
    # Aplicar UMAP
    umap_model = umap.UMAP(n_components=n_dim, random_state=42)
    reduced_embeddings = umap_model.fit_transform(embeddings)
    
    # Reemplazar la columna de embeddings
    df['embedding'] = reduced_embeddings.tolist()
    
    print(f"UMAP aplicado exitosamente. Dimensiones reducidas a {n_dim}.")
    return df
df_red = aplicar_umap(df, n_dim=200)


**Verificación de formatos y estructura previo a la indexación**

In [6]:
# Verificar tipo de dato y longitud de cada embedding
df['embedding_type'] = df['embedding'].apply(lambda x: type(x).__name__)
df['embedding_length'] = df['embedding'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Mostrar un resumen de los tipos de datos encontrados
print(df['embedding_type'].value_counts())

#  Mostrar un resumen de las longitudes de embeddings encontradas
print(df['embedding_length'].value_counts())


embedding_type
list    490965
Name: count, dtype: int64
embedding_length
384    490965
Name: count, dtype: int64


In [ ]:
# Verificar si todos los embeddings son listas de floats
error_count = 0

for index, row in df.iterrows():
    embedding = row['embedding']
    if not isinstance(embedding, list) or not all(isinstance(x, float) for x in embedding):
        print(f" Registro inválido en el índice {index}. Tipo de embedding: {type(embedding)}")
        error_count += 1

if error_count == 0:
    print(" Todos los registros tienen embeddings válidos.")
else:
    print(f" Se encontraron {error_count} registros inválidos.")


🎉 Todos los registros tienen embeddings válidos.


In [ ]:
# Comprobar configuración del índice
index_info = es.indices.get_mapping(index=index_name)
index_info_dict = index_info.body if hasattr(index_info, 'body') else index_info  # Convertir a diccionario si es necesario
print(json.dumps(index_info_dict, indent=2))


{
  "amazon_products_embeddings": {
    "mappings": {
      "properties": {
        "embedding": {
          "type": "dense_vector",
          "dims": 384,
          "index": true,
          "similarity": "cosine",
          "index_options": {
            "type": "int8_hnsw",
            "m": 16,
            "ef_construction": 100
          }
        },
        "link": {
          "type": "keyword"
        },
        "main_category": {
          "type": "keyword"
        },
        "name": {
          "type": "text"
        },
        "ratings": {
          "type": "float"
        },
        "sub_category": {
          "type": "keyword"
        }
      }
    }
  }
}


### **Creación del indice con Embeddings**

**Mapping**

In [ ]:

# Establecer la conexión con Elasticsearch
es = Elasticsearch("http://localhost:9200")
embedding_dim = len(df['embedding'].iloc[0])

# Nombre del índice
index_name = "amazon_products_embeddings"

# Configuración del índice
index_config = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    },
    "mappings": {
        "properties": {
            "name": {"type": "text"},
            "main_category": {"type": "keyword"},
            "sub_category": {"type": "keyword"},
            "link": {"type": "keyword"},
            "ratings": {"type": "float"},
            "embedding": {
                "type": "dense_vector",
                "dims": embedding_dim   # se asigna dinámicamente
            }
        }
    }
}

#  Eliminar el índice si ya existe
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' eliminado exitosamente.")

# Crear el índice nuevamente con la nueva configuración
es.indices.create(index=index_name, body=index_config)
print(f" Índice '{index_name}' creado exitosamente con dimensión {embedding_dim}.")


Índice 'amazon_products_embeddings' eliminado exitosamente.
 Índice 'amazon_products_embeddings' creado exitosamente con dimensión 384.


**Indexing**

In [ ]:
from elasticsearch import helpers
import json

def indexar_datos_embeddings_v2(es, df, index_name, chunk_size=2500, embedding_dim=384):
    # Mostrar cantidad de filas antes de la limpieza
    print(f" Total de filas antes de limpiar: {len(df)}")
    
    # Limpiar el DataFrame y reemplazar NaNs por None
    df_cleaned = df[['name', 'main_category', 'sub_category', 'ratings', 'no_of_ratings', 
                     'discount_price_dolares', 'actual_price_dolares', 'embedding']].copy()
    
    # Filtrar filas con embeddings no válidos o incompletos
    df_cleaned = df_cleaned[df_cleaned['embedding'].apply(lambda x: isinstance(x, list) and len(x) == embedding_dim)]
    df_cleaned = df_cleaned.where(pd.notna(df_cleaned), None)
    
    # Mostrar cantidad de filas después de la limpieza
    print(f" Total de filas después de limpiar: {len(df_cleaned)}")
    
    # Conversión explícita de columnas
    df_cleaned['ratings'] = pd.to_numeric(df_cleaned['ratings'], errors='coerce').fillna(0)
    df_cleaned['no_of_ratings'] = pd.to_numeric(df_cleaned['no_of_ratings'], errors='coerce').fillna(0)
    df_cleaned['discount_price_dolares'] = pd.to_numeric(df_cleaned['discount_price_dolares'], errors='coerce').fillna(0)
    df_cleaned['actual_price_dolares'] = pd.to_numeric(df_cleaned['actual_price_dolares'], errors='coerce').fillna(0)

    # Índices de documentos fallidos
    failed_documents = []

    # Indexar en lotes
    total_indexed = 0
    total_failed = 0

    for start_idx in range(0, len(df_cleaned), chunk_size):
        end_idx = min(start_idx + chunk_size, len(df_cleaned))
        batch = df_cleaned.iloc[start_idx:end_idx].to_dict(orient="records")
        
        # Crear el formato adecuado para la indexación
        bulk_data = [
            {
                "_index": index_name,
                "_source": {
                    "name": record["name"],
                    "main_category": record["main_category"],
                    "sub_category": record["sub_category"],
                    "ratings": record["ratings"],
                    "no_of_ratings": record["no_of_ratings"],
                    "discount_price_dolares": record["discount_price_dolares"],
                    "actual_price_dolares": record["actual_price_dolares"],
                    "embedding": record["embedding"]
                }
            }
            for record in batch
        ]
        
        # Intentar indexar y capturar errores
        try:
            success, failed = helpers.bulk(es, bulk_data, stats_only=True)
            total_indexed += success
            total_failed += failed
            #print(f" Lote {start_idx // chunk_size + 1} indexado correctamente: {success} documentos.")
        
        except Exception as e:
            #print(f" Error al indexar el lote {start_idx // chunk_size + 1}: {e}")
            failed_documents.append({
                "lote": start_idx // chunk_size + 1,
                "error": str(e)
            })
    
    # Mostrar resultados finales
    print(f"\n Indexación completada. Total indexados: {total_indexed}. Total fallidos: {total_failed}.")
    
    # Guardar errores en un archivo JSON si existen
    if failed_documents:
        with open('errores_indexacion.json', 'w') as f:
            json.dump(failed_documents, f, indent=4)
        print(" Los errores se han guardado en 'errores_indexacion.json'.")



In [36]:
indexar_datos_embeddings_v2(es, df, index_name, chunk_size=2500, embedding_dim=384)

 Total de filas antes de limpiar: 490965
 Total de filas después de limpiar: 490965

 Indexación completada. Total indexados: 190000. Total fallidos: 0.
 Los errores se han guardado en 'errores_indexacion.json'.


### **Función para realizar consultas usando metricas de similitud**

In [ ]:

def buscar_productos_similares(es, index_name, product_name, top_k=5):
    """
    Realiza una búsqueda en Elasticsearch usando embeddings para encontrar productos similares.
    (Esta función utiliza cosineSimilarity de Elasticsearch para mejor rendimiento).
    
    Args:
        es (Elasticsearch): Conexión a Elasticsearch.
        index_name (str): Nombre del índice en Elasticsearch.
        product_name (str): Nombre del producto a buscar.
        top_k (int): Número de productos similares a devolver.
        
    Returns:
        list: Lista de productos similares encontrados.
    """
    # Buscar el producto original por nombre en Elasticsearch
    query = {
        "query": {
            "match": {
                "name": product_name
            }
        }
    }
    
    response = es.search(index=index_name, body=query)
    
    if not response["hits"]["hits"]:
        print(f" Producto '{product_name}' no encontrado en el índice.")
        return []
    
    # Obtener el embedding del producto encontrado
    producto = response["hits"]["hits"][0]["_source"]
    product_embedding = producto["embedding"]
    
    # Construir la consulta de similitud con embeddings
    search_body = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": product_embedding}
                }
            }
        }
    }

    # Realizar la búsqueda en Elasticsearch
    similar_products = es.search(index=index_name, body=search_body)
    
    # Procesar resultados
    results = [
        {
            "name": hit["_source"]["name"],
            "main_category": hit["_source"]["main_category"],
            "sub_category": hit["_source"]["sub_category"],
            "ratings": hit["_source"]["ratings"],
            "no_of_ratings": hit["_source"]["no_of_ratings"],
            "discount_price_dolares": hit["_source"]["discount_price_dolares"],
            "actual_price_dolares": hit["_source"]["actual_price_dolares"],
            "score": hit["_score"]
        }
        for hit in similar_products["hits"]["hits"]
    ]
    
    return results



**Prueba de funcionalidad**

In [ ]:
resultados = buscar_productos_similares(es, index_name="amazon_products_embeddings", product_name="Echo Dot", top_k=5)
for producto in resultados:
    print(f"Producto: {producto['name']}, Score: {producto['score']}")

In [ ]:
resultados = buscar_productos_similares(es, index_name="amazon_products_embeddings", product_name="AirPods", top_k=5)
for producto in resultados:
    print(f"Producto: {producto['name']}, Score: {producto['score']}")


In [ ]:
resultados = buscar_productos_similares(es, index_name="amazon_products_embeddings", product_name="Yoga Mat", top_k=5)
for producto in resultados:
    print(f"Producto: {producto['name']}, Score: {producto['score']}")
